# <center>R4.B.10 - Cryptographie et sécurité <br> TP1 - Protocole de sécurité WEP <center>


_Tom Ferragut_
    
_IUT de Vannes, BUT Informatique_

In [1]:
import numpy
import random

## 1 - Simulation du protocole

Le protocole WEP (Wired Equivalent Privacy) a été l'une des premières tentatives de sécurisation des réseaux sans fil. Malheureusement, des faiblesses majeures ont été découvertes, le rendant vulnérable à diverses attaques. Dans cette première partie du TP, nous simulerons le fonctionnement de la sécurité WEP en générant des discussions entre des périphériques connectés à un réseau.

Voici quelques fonctions que vous pourrez utiliser dans les questions suivantes.

In [2]:
def generate_key(k, WEP_key, iv):
    # Générer grâce à la clé WEP et à l'IV une keystream aléatoire de k bits
    random.seed( iv+WEP_key )
    return ''.join(random.choice('01') for _ in range(k))

def encrypt(message, keystream):
    # Chiffrer le message en utilisant une keystream
    encrypted_message = ''
    for i in range(len(message)):
        encrypted_message += chr(ord(message[i]) ^ ord(keystream[i % len(keystream)]))
    return encrypted_message

def decrypt(encrypted_message, keystream):
    # Déchiffrer le message en utilisant une clé généré par WEP et l'IV
    decrypted_message = encrypt(encrypted_message,keystream)
    return decrypted_message


>__Question 1 :__ Implémenter une fonction `discussion` prenant en paramètre un entier `n` et renvoyant une liste de `n` couples $[~[IV\_1,Message\_1]~,~...~,~[IV\_n,Message\_n]~]$ contenant chacun un IV (de 8 bit) aléatoire ainsi qu'un message aléatoire de 12 caractères commençant par les quatres caractères IPv4.
>
>Pour générer des caractères aléatoires vous pouvez utiliser la fonction __random.choice__.

In [18]:
def discussion(n):
    disc = []
    for i in range(n) :
        mess = "IPv4" + ''.join([random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz') for k in range(8)])
        iv = ''.join([random.choice('01') for k in range(8)])
        disc.append((iv,mess))
    return  disc

# Test de la fonction avec n=5
n = 5
liste_messages = discussion(n)
print("Liste de messages générée avec n =", n, ":")
for message in liste_messages : 
    print(message)

Liste de messages générée avec n = 5 :
('11000001', 'IPv4zhaGdBEC')
('00010101', 'IPv4pgRKMPNV')
('00010101', 'IPv4AbblPphj')
('11000000', 'IPv4xYDksMCu')
('00101010', 'IPv4EOQhEIdg')


>__Question 2 :__ Implémenter une fonction `ecoute` prenant en paramètre un entier `n` et une clé WEP `WEP_key` et renvoyant une liste de `n` couples $[~[IV\_1,Message\_ crypté\_1]~,~...~,~[IV\_n,Message\_crypté\_n]~]$ contenant chacun un IV (de 8 bit) aléatoire ainsi que le message crypté provenant d'une discussion générée dans la fonction `discussion`. 
>
>Chaque message est chiffré par la _keystream_ généré par `generate_key` grâce à son IV et une clé WEP commune.

In [22]:
def ecoute(n,WEP_key):
    cipher = []
    disc = discussion(n)
    for iv,message in disc : 
        keystream = generate_key(len(message)*8, WEP_key, iv)
        ciphertext = encrypt(message, keystream)
        cipher.append((iv,ciphertext))
    return cipher


# Test de la fonction avec n=5
n = 5
WEP_key = ''.join(random.choice('01') for _ in range(15))


liste_messages = ecoute(n,WEP_key)
print("Liste de messages chiffrés générés avec n =", n, ":")
for m in liste_messages : 
    print(m)

Liste de messages chiffrés générés avec n = 5 :
('10001000', 'y`G\x04CF|Fgvw_')
('11110111', 'xaF\x05WeKWeYia')
('01100110', 'xaF\x04B^F|BHHS')
('11010100', 'x`F\x04T]tega`q')
('10111111', 'x`G\x05y`B~SkIz')


Une première faiblesse du protocole WEP est le fait qu'il transporte en claire une partie de la clé : l'__IV__. Cela est sencé créer différentes _keystream_ pour différents paquets envoyés, rendant la tâche de décryptage plus difficile. Cependant, même pour une un IV de 8bits ($2^8=256$ possibilités), deux messages ayant un même IV arrive très souvent.

>__Question 3 :__ Implémenter une fonction `find_double` qui écoute un réseau WEP puis identifie et retourne deux messages cryptés ayant le même IV.

In [54]:
def find_double(messages):
    
    doubles = {}
    for iv,mess in messages : 
        if iv in doubles.keys() : 
            doubles[iv].append(mess)
        else : 
            doubles[iv] = [mess]
    
    # Trie
    lst = {}
    for iv,doublons in doubles.items() : 
        if len(doublons) >= 2 : 
            lst[iv] = doublons
    return lst

# Test de la fonction avec une liste de messages
n=30
messages = ecoute(n,WEP_key)
resultat = find_double(messages)
print("Double(s) trouvé(s) :")
for key in resultat : 
    print(f"{key} : {resultat[key]}")

Double(s) trouvé(s) :
11100101 : ['x`F\x04]YUi\\H`r', 'x`F\x04\\hTwz|gv']
11010011 : ['y`F\x04hsByeR|{', 'y`F\x04T{AVf@Ht']
11001011 : ['x`F\x05gtHgfzbE', 'x`F\x05\\wac\\@aW']


## 2 - Attaque par fragmentation de paquets


La deuxième partie de ce TP se concentrera sur une attaque spécifique exploitant une faiblesse du protocole WEP liée à la fragmentation des paquets. Cette attaque permet de retrouver la _keystream_ utilisée pour crypter les messages, compromettant ainsi la sécurité du réseau.

Grâce à deux messages envoyés avec un même IV, il est possible de trouver les premiers bits de la _keystream_. Une autre solution pour obtenir les premiers éléments de la clé est d'utiliser la structure des paquets envoyés, par exemple le fait qu'ici chaque message commence par 'IPv4'. 

Voici une fonction `find_key` donnant la clé chiffrante à partir du message en clair et du message chiffré.

>__Question 4 :__ Implémenter une fonction `find_key` qui donne la clé à partir du message en clair et du message crypté.

In [56]:
# M xor C = KeyStream (ou une partie)
# Evidemment, il faut connaitre M (ou une partie) et C (ou une partie)
def find_key(plaintext, encrypted_message):
    
    # Effectuer une opération XOR entre le message en clair et le message crypté pour obtenir la partie de la keystream
    keystream = ''
    for i in range(len(plaintext)):
        keystream += chr(ord(plaintext[i]) ^ ord(encrypted_message[i]))
    return keystream


key = '0100'

message1 = 'IPv4'
encrypted_message1 = encrypt(message1, key)

found_key = find_key(message1, encrypted_message1)

print("Clé récupérée :", found_key)


Clé récupérée : 0100


Nous allons maintenant utiliser le principe de fragmentation de paquets utilisé dans le protocole WEP. Un message n'est pas envoyé en un seul bloc, mais découpé en morceaux plus petits envoyés séparéments.

>__Question 4 :__ Implémenter une fonction `fragment_packet` qui prend en argument une paquet `packet` (une message) et une taille de fragment `fragment_size` puist qui renvoie une liste contenant, dans l'ordre, les morceaux de taille `fragment_size` composant le `packet`.

In [58]:
def fragment_packet(packet, fragment_size):
    fragments = []
    for k in range(0,len(packet),fragment_size) : 
        fragments.append(packet[k:k+fragment_size])
    
    return fragments

fragment_packet('abcdefghigklmnopqurstuvwxyz', 5)

['abcde', 'fghig', 'klmno', 'pqurs', 'tuvwx', 'yz']

__La première faiblesse__ de sécurité est que chaque fragment du paquet est chiffré avec une même sous-partie du keystream, une clé plus petite que l'on appellera `small_key`.

>__Question 5 :__ Implémenter une fonction `encrypting_fragments` qui prend en argument une une liste de fragments `fragments` et une clé `small_key` et qui renvoie la liste des fragments chiffrés par la clé `small_key`.

In [75]:
def encrypting_fragments(fragments, small_key):
    ciphers = []
    for frag in fragments : 
        ciphertext = encrypt(frag,small_key)
        ciphers.append(ciphertext)
    return ciphers

fragments = fragment_packet('abcdefghigklmnopqurstuvwxyz', 4)
encrypting_fragments(fragments, '10010101')

['PRSU', 'TVWY', 'XW[]', '\\^_A', '@EBB', 'EEFF', 'IIJ']

__La deuxième faiblesse__ du protocole WEP, celle nous permettant de retrouver le _keystream_ complet, est le passage des fragments par l'_Access Point_ (AP). Cet AP décrypte localement les fragments, réassemble le message en clair et renvoie le message chiffré par le _keystream_ complet.

In [95]:
# Access point
def AP(fragmented_encrypted_message,small_key): 
    #l'AP décrypte les fragments du message
    decrypted_fragments=[encrypt(fragment, small_key) for fragment in fragmented_encrypted_message]
    #l'AP reconstruit le message et le renvoie crypté
    reconstructed_message = ''.join(decrypted_fragments)
    #l'AP recrypte le message avec la keystream
    encrypted_message = encrypt(reconstructed_message, keystream)
    
    return encrypted_message

Nous avons maintenant l'ensemble des éléments pour décrypté le message et ainsi obtenir le _keystream_ complet, voyez-vous comment ?

>__Question 6 :__ Implémenter une fonction `recover_keystream_from_fragments` qui à partir d'une liste de fragments cryptés, trouve et renvoie le message en clair correspondant ainsi que la `keystream` utilisée pour crypter le message.

In [101]:
# Si la small_key est connue
# Si le fragement_size est connu
# On peut récuprer le plaintext
# plain XOR cipher = keystream
def recover_key_from_fragments(message_crypted_fragments,fragment_size):
    begin = "IPv4"
    smallkey = ''
    for i,char in enumerate(message_crypted_fragments[0]):
        smallkey += chr(ord(begin[i]) ^ ord(char))
    
    decrypted_fragments=[encrypt(fragment, smallkey) for fragment in message_crypted_fragments]
    plaintext = ''.join(decrypted_fragments)
    
    fullcipher = AP(message_crypted_fragments,smallkey)
    keystream = ""
    for i in range(len(fullcipher)):
        keystream += chr(ord(plaintext[i]) ^ ord(fullcipher[i]))
    
    return [plaintext,keystream]


Vous pouvez tester votre méthode en exécutant la cellule suivante.

In [102]:
### Exemple d'utilisation ###

## Données non accessibles ##

message = "IPv4: Contenu du contrôle terminal de la ressource Cryptographie et sécurité"
WEP_key = ''.join(random.choice('01') for _ in range(15))
iv = ''.join(random.choice('01') for _ in range(8))  # Générer un IV aléatoire
keystream = generate_key(len(message),WEP_key,iv)
small_key=keystream[:4]

fragment_size=4

## Données accessibles ##

#Ecoute du message (fragmenté) crypté
encrypted_fragments = encrypting_fragments(fragment_packet(message, fragment_size),keystream[:fragment_size])

#Appel de la fonction de décryptage
recovered_key_from_fragments = recover_key_from_fragments(encrypted_fragments,fragment_size)

## Affichage du résultat ##

print('''--- Données non accessibles ---
''')
print("Message en clair :", message)
print("Clé WEP :", WEP_key)
print("IV :", iv)
print("Keystream cryptant le message :", keystream,''' 

--- Données accessibles ---
''')

print("Liste des communications écoutés :",encrypted_fragments)
print("Keystream récupéré :", recovered_key_from_fragments[1])
print('Message en clair :', recovered_key_from_fragments[0])

--- Données non accessibles ---

Message en clair : IPv4: Contenu du contrôle terminal de la ressource Cryptographie et sécurité
Clé WEP : 110010110001101
IV : 11111100
Keystream cryptant le message : 1011100001101000110101100101111010000110011011100000011101110111111001100001  

--- Données accessibles ---

Liste des communications écoutés : ['x`G\x05', '\x0b\x10r^', '_DT_', 'D\x10UD', '\x11S^_', 'EBÅ]', 'T\x10ET', 'C]X_', 'P\\\x11U', 'T\x10]P', '\x11BTB', 'B_DC', 'RU\x11r', 'CIAE', '^WCP', 'AXXT', '\x11UE\x11', 'BÙRD', 'CYEØ']
Keystream récupéré : 1011100001101000110101100101111010000110011011100000011101110111111001100001
Message en clair : IPv4: Contenu du contrôle terminal de la ressource Cryptographie et sécurité
